In [1]:
import pandas as pd

In [2]:
df_d15 = pd.read_excel('/Users/adamzsolt/Desktop/Thesis/RESTART/Datasets/Blanchardstown.xltx')

In [3]:
df_dl = pd.read_excel('/Users/adamzsolt/Desktop/Thesis/RESTART/Datasets/Dun_Laoghaire.xltx')

In [4]:
df_port = pd.read_excel('/Users/adamzsolt/Desktop/Thesis/RESTART/Datasets/Dublin_Port.xltx')

In [5]:
df_stjr = pd.read_excel('/Users/adamzsolt/Desktop/Thesis/RESTART/Datasets/St_Johns_Road.xltx')

In [6]:
df_tal = pd.read_excel('/Users/adamzsolt/Desktop/Thesis/RESTART/Datasets/Tallagh.xltx')

In [7]:
df_d15.head()

,"Air Quality Levels at Blanchardstown, Dublin 15",Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Date and Time,NO2,PM10,PM2.5
1,2020-07-01 00:00:00,1.77,13.64,8
2,2020-07-01 01:00:00,2.12,10.91,7.48
3,2020-07-01 02:00:00,1.59,13.06,6.78
4,2020-07-01 03:00:00,0.58,17.97,9.55


In [8]:
df_d15['Location']= "Blanchardstown"

In [9]:
df_dl['Location']= "DunLaoghaire"

In [10]:
df_port['Location']= "DublinPort"

In [11]:
df_stjr['Location']= "StJohnsRoad"

In [12]:
df_tal['Location']= "Tallagh"

In [13]:
def preprocess_data(df):
    
    # Renaming the first column to 'Date'
    df.columns.values[0] = 'Date'
    
    # Renaming columns
    df.rename(columns={'Unnamed: 1': 'NO2', 'Unnamed: 2': 'PM10', 'Unnamed: 3': 'PM2.5'}, inplace=True)
    
    # Dropping the first observation (first row)
    df.drop(0, inplace=True)
    
    # Resetting the index after dropping the first row
    df.reset_index(drop=True, inplace=True)
    
    return df

In [14]:
df_d15 = preprocess_data(df_d15)

In [15]:
df_dl = preprocess_data(df_dl)

In [16]:
df_port = preprocess_data(df_port)

In [17]:
df_stjr = preprocess_data(df_stjr)

In [18]:
df_tal = preprocess_data(df_tal)

In [19]:
# Concatenating DataFrames vertically based on 'Date' column
dfs = pd.concat([df_d15, df_dl, df_port, df_stjr, df_tal], ignore_index=True)
dfs.drop(dfs.columns[-1], axis=1, inplace=True)
dfs.head()

,Date,NO2,PM10,PM2.5,Location
0,2020-07-01 00:00:00,1.77,13.64,8,Blanchardstown
1,2020-07-01 01:00:00,2.12,10.91,7.48,Blanchardstown
2,2020-07-01 02:00:00,1.59,13.06,6.78,Blanchardstown
3,2020-07-01 03:00:00,0.58,17.97,9.55,Blanchardstown
4,2020-07-01 04:00:00,-0.61,17.97,11.05,Blanchardstown


In [20]:
dfs.shape

(149874, 5)

In [21]:
dfs.isnull().sum()

Date           1
NO2         1178
PM10        2103
PM2.5       2022
Location       0
dtype: int64

In [22]:
from sklearn.impute import SimpleImputer

# Convert the columns to float type, handling NaN values
dfs[['NO2', 'PM2.5', 'PM10']] = dfs[['NO2', 'PM2.5', 'PM10']].apply(pd.to_numeric, errors='coerce')

# Impute missing values using median strategy
imputer = SimpleImputer(strategy='median')
dfs[['NO2', 'PM2.5', 'PM10']] = imputer.fit_transform(dfs[['NO2', 'PM2.5', 'PM10']])

dfs.dropna(subset=['Date'], inplace=True)

In [23]:
dfs.isnull().sum()

Date        0
NO2         0
PM10        0
PM2.5       0
Location    0
dtype: int64

In [24]:
dfs.describe()

,NO2,PM10,PM2.5
count,149873.000000,149873.000000,149873.000000
mean,21.575163,10.140529,9.251968
std,18.417078,12.739083,10.081731
min,-15.400000,-11.880000,0.000000
25%,7.390000,3.960000,3.700000
50%,16.260000,8.170000,6.220000
75%,31.560000,13.540000,11.040000
max,339.380000,1151.360000,460.050000


In [25]:
dfs['Date'] = pd.to_datetime(dfs['Date'])

dfs['Year'] = dfs['Date'].dt.year.astype(int)
dfs['Month'] = dfs['Date'].dt.month.astype(int)
dfs['Day'] = dfs['Date'].dt.day.astype(int)
dfs['Hour'] = dfs['Date'].dt.hour.astype(int)

dfs = dfs[['Date', 'Year', 'Month', 'Day', 'Hour', 'Location', 'NO2', 'PM2.5', 'PM10']]

In [26]:
dfs.head()

,Date,Year,Month,Day,Hour,Location,NO2,PM2.5,PM10
0,2020-07-01 00:00:00,2020,7,1,0,Blanchardstown,1.77,8.00,13.64
1,2020-07-01 01:00:00,2020,7,1,1,Blanchardstown,2.12,7.48,10.91
2,2020-07-01 02:00:00,2020,7,1,2,Blanchardstown,1.59,6.78,13.06
3,2020-07-01 03:00:00,2020,7,1,3,Blanchardstown,0.58,9.55,17.97
4,2020-07-01 04:00:00,2020,7,1,4,Blanchardstown,-0.61,11.05,17.97
